In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
#import pickle
from scipy.spatial import cKDTree
from SCTransform import SCTransform

import warnings
import time
warnings.filterwarnings('ignore')
#export PYTHONWARNINGS='ignore:Multiprocessing-backed parallel loops:UserWarning'
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"



In [2]:
def create_directory(outputFolder):
    answer=os.path.isdir(outputFolder)
    if answer==True:
        pass
    else:
        os.mkdir(outputFolder)


def find_index(sp_genename,sc_genename):
    index_sc=[]
    index_sp=[]
    d={}
    for j in range(len(sc_genename)):
        name=sc_genename[j]
        d[name]=j

    for i in range(len(sp_genename)):
        name=sp_genename[i]
        try:
            d[name]
            flag=1
        except KeyError:
            flag=0
        if flag==1:
            index_sc.append(d[name])
            index_sp.append(i)
    return index_sp,index_sc


In [3]:
scdatapath='./Intestine_sc_data/DanaGroupData/'
spdatapath='./OriginalSpatialData/'

ad_sp_ori=sc.read(spdatapath+'gene_by_cell.csv').transpose()

ad_sc_li=sc.read_h5ad(scdatapath+'LI_adata_paper.h5ad') #.transpose()
ad_sc_si=sc.read_h5ad(scdatapath+'SI_adata_paper.h5ad') #.transpose()


#ad_sp_ori.var_names_make_unique()
#ad_sc_ori.var_names_make_unique()

#sc.pp.filter_cells(ad_sp_ori, min_counts=2)

In [4]:
v1=ad_sc_li.X.todense()
v2=ad_sc_si.X.todense()
print(v1)
print(v2)

[[0.         0.         0.         ... 0.69514954 0.         0.        ]
 [0.         0.         0.         ... 0.7007411  0.         0.        ]
 [0.08411071 0.         0.         ... 0.5820725  0.16169243 0.        ]
 ...
 [0.         0.         1.9676502  ... 1.9676502  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0.         0.         0.         ... 0.05349241 0.         0.        ]
 [0.         0.         0.         ... 0.05884708 0.         0.        ]
 [0.         0.         0.         ... 0.06043441 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 2.548312   0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [8]:
ad_sp_ori.X

array([[ 0.,  0.,  0., ...,  0.,  0.,  9.],
       [ 0., 39.,  0., ...,  0.,  0.,  5.],
       [ 0., 49.,  0., ...,  0.,  0.,  4.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [5]:
ad_sc_li.X = ad_sc_li.layers['counts']
ad_sc_si.X = ad_sc_si.layers['counts']

In [6]:
v1=ad_sc_li.X.todense()
v2=ad_sc_si.X.todense()
print(v1)
print(v2)

[[ 0.  0.  0. ... 14.  0.  0.]
 [ 0.  0.  0. ... 12.  0.  0.]
 [ 1.  0.  0. ...  9.  2.  0.]
 ...
 [ 0.  0.  1. ...  1.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 2. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
print(ad_sc_li)
print(ad_sc_si)
print(ad_sp_ori)

AnnData object with n_obs × n_vars = 5163 × 19706
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'predicted_doublet', 'doublet_score', 'Phenograph_cluster_k15_dbRM', 'coarse_cluster_dbRM', 'cell_state'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Phenograph_cluster_k15_dbRM_colors', 'cell_state_colors', 'coarse_cluster_dbRM_colors', 'dendrogram_Phenograph_cluster_k15_dbRM', 'hvg', 'neighbors', 'pca', 'test_elbo', 'test_epoch', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
AnnData object with n_obs × n_vars = 2239 × 18573
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts

In [10]:
sp_genename=   ad_sp_ori.var_names.to_numpy()
sc_genename_li=ad_sc_li.var_names.to_numpy()
sc_genename_si=ad_sc_si.var_names.to_numpy()

index_sp,index_sc=find_index(sp_genename,sc_genename_li)
ad_li=ad_sc_li[:,index_sc].copy()
ad_sp_li=ad_sp_ori[:,index_sp].copy()

print(sp_genename[index_sp])
print(sc_genename_li[index_sc])


index_sp,index_sc=find_index(sp_genename,sc_genename_si)
ad_si=ad_sc_si[:,index_sc].copy()
ad_sp_si=ad_sp_ori[:,index_sp].copy()

#print(sp_genename[index_sp])
#print(sc_genename_si[index_sc])

['Acsl1' 'Acta2' 'Ada' 'Adgrd1' 'Adgrf5' 'Adra1a' 'Adra1b' 'Adra2a'
 'Adra2c' 'Adrb1' 'Adrb2' 'Agr3' 'Apob' 'Avil' 'C1qb' 'C1qc' 'Cadm1'
 'Ccl9' 'Ccnl2' 'Ccr7' 'Cd14' 'Cd27' 'Cd34' 'Cd3d' 'Cd3e' 'Cd4' 'Cd44'
 'Cd79a' 'Cd79b' 'Cd8a' 'Cd8b1' 'Chrm1' 'Chrm2' 'Chrm3' 'Clca3b' 'Clec4a2'
 'Clec9a' 'Cmc1' 'Cps1' 'Cspg4' 'Cxcr5' 'Cysltr1' 'Cysltr2' 'Drd1' 'Drd3'
 'Drd4' 'Elavl3' 'Etv1' 'Fcgr4' 'Fcna' 'Ffar1' 'Ffar2' 'Ffar3' 'Ffar4'
 'Foxp3' 'Fzd1' 'Fzd10' 'Fzd2' 'Fzd3' 'Fzd4' 'Fzd5' 'Fzd6' 'Fzd7' 'Fzd8'
 'Fzd9' 'Galr1' 'Galr2' 'Galr3' 'Gipr' 'Glp1r' 'Glp2r' 'Gpbar1' 'Gper1'
 'Gpr119' 'Gpr174' 'Gpr18' 'Gpr31b' 'Gpr34' 'Gpr55' 'Gpr84' 'Gzmb'
 'H2-Eb1' 'Hhex' 'Hmgb2' 'Hrh1' 'Hrh2' 'Hrh3' 'Htr1b' 'Htr1f' 'Htr2b'
 'Htr4' 'Htr5b' 'Htr7' 'Icos' 'Ifnar1' 'Ifng' 'Ighd' 'Il10' 'Il3ra' 'Il4'
 'Il5' 'Il7r' 'Isl1' 'Itgae' 'Itgam' 'Jaml' 'Kiss1r' 'Kit' 'Klk1' 'Krt7'
 'Lamp3' 'Lct' 'Lgr5' 'Lmx1a' 'Lpar1' 'Lpar2' 'Lpar3' 'Lpar4' 'Lpar5'
 'Lpar6' 'Lrmp' 'Ltb4r1' 'Ltb4r2' 'Ly6g' 'Maged2' 'Maoa' 'Marcksl1'
 'Mbo

In [11]:
print(ad_li)
print(ad_sp_li)
print(ad_si)
print(ad_sp_si)

AnnData object with n_obs × n_vars = 5163 × 194
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'predicted_doublet', 'doublet_score', 'Phenograph_cluster_k15_dbRM', 'coarse_cluster_dbRM', 'cell_state'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Phenograph_cluster_k15_dbRM_colors', 'cell_state_colors', 'coarse_cluster_dbRM_colors', 'dendrogram_Phenograph_cluster_k15_dbRM', 'hvg', 'neighbors', 'pca', 'test_elbo', 'test_epoch', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
AnnData object with n_obs × n_vars = 7416 × 194
AnnData object with n_obs × n_vars = 2239 × 188
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'n_genes', 'n_gen

In [12]:
sct_ad_sp_li = SCTransform(ad_sp_li,min_cells=1,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
                        bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sp_li.write_h5ad('scTransform_spatial_li.h5ad')

In [13]:
sct_ad_sp_si = SCTransform(ad_sp_si,min_cells=1,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
                        bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sp_si.write_h5ad('scTransform_spatial_si.h5ad')

In [14]:
celname1=ad_sc.obs_names.to_numpy()
celname2=ad_sp.obs_names.to_numpy()
s1=np.unique(celname1)
s2=np.unique(celname2)
print(len(celname1),len(celname2))
print(len(s1),len(s2))

2239 7416
2239 7416


In [11]:
#sc.pp.filter_cells(ad_sc, min_counts=1)
#print(ad_sc)

In [14]:
sct_ad_sc_li = SCTransform(ad_li,min_cells=1,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
		                bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sc_li.write_h5ad('scTransform_singleCell_li.h5ad')

In [15]:
sct_ad_sc_si = SCTransform(ad_si,min_cells=1,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
		                bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sc_si.write_h5ad('scTransform_singleCell_si.h5ad')